# Lab 11: Grid Localization using Bayes Filter (Real Robot)

### <span style="color:rgb(0,150,0)">It is recommended that you close any heavy-duty applications running on your system while working on this lab.</span>

### <span style="color:rgb(0,150,0)">The notebook only provides skeleton code for you to integrate the Localization class with the Real Robot.</span>

<hr>

In [108]:
%load_ext autoreload
%autoreload 2

import traceback
from notebook_utils import *
from Traj import *
import asyncio
import pathlib
import os
from utils import load_config_params
from localization_extras import Localization

# The imports below will only work if you copied the required ble-related python files 
# into the notebooks directory
from ble import get_ble_controller
from base_ble import LOG
from cmd_types import CMD
import numpy as np

# Setup Logger
LOG = get_logger('demo_notebook.log')
LOG.propagate = False

# Init GUI and Commander
gui = GET_GUI()
cmdr = gui.launcher.commander

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
2025-05-14 13:22:21,752 | INFO     |: Logger demo_notebook.log already initialized.
2025-05-14 13:22:21,753 | INFO     |: GUI is already running. Shutdown notebook to force restart the GUI.


In [110]:
# Start the plotter
START_PLOTTER()

2025-05-14 13:22:27,146 | INFO     |: Creating New Plotter Process


In [109]:
STOP_PLOTTER()

2025-05-14 13:22:25,352 | INFO     |: Plotter is stopped


# The RealRobot class
Define the RealRobot class in the code cell below, based on the documentation and your real robot communication protocol. <br>
This class is used by the **Localization** class to communicate with the real robot. <br>
More specifically, the **Localization** class utilizes the **RealRobot's** member function **perform_observation_loop()** to get the 18 sensor readings and store them in its member variable **obs_range_data**, which is then utilized in the update step.

In [120]:
import csv
import datetime
import asyncio


class RealRobot():
    """A class to interact with the real robot
    """

    def __init__(self, commander, ble):
        # Load world config
        self.world_config = os.path.join(
            str(pathlib.Path(os.getcwd()).parent), "config", "world.yaml")

        self.config_params = load_config_params(self.world_config)

        # Commander to commuincate with the Plotter process
        # Used by the Localization module to plot odom and belief
        self.cmdr = commander

        # ArtemisBLEController to communicate with the Robot
        self.ble = ble

        self.is_done_collecting = False
        self.results = []
        self.ble.start_notify(
            self.ble.uuid['RX_STRING'], self.notification_handler)

    def notification_handler(self, uuid, byte_array):
        result = self.ble.bytearray_to_string(byte_array)
        if result == "DONE":
            LOG.info("Done Notification Received")
            self.is_done_collecting = True
        else:
            LOG.info("Data Notification Received...processing")
            time, imu, pwm, tof = self.ble.bytearray_to_string(
                byte_array).split('|')
            self.results.append(
                [float(time), float(imu), int(pwm), float(tof)])

    def get_pose(self):
        """Get robot pose based on odometry

        Returns:
            current_odom -- Odometry Pose (meters, meters, degrees)
        """
        current_odom, current_gt = self.cmdr.get_pose()
        return self.get_pose_in_degrees(current_odom), self.get_pose_in_degrees(current_gt)

    async def perform_observation_loop(self, rot_vel=120):
        """Perform the observation loop behavior on the real robot, where the robot does  
        a 360 degree turn in place while collecting equidistant (in the angular space) sensor
        readings, with the first sensor reading taken at the robot's current heading. 
        The number of sensor readings depends on "observations_count"(=18) defined in world.yaml.

        Keyword arguments:
            rot_vel -- (Optional) Angular Velocity for loop (degrees/second)
                        Do not remove this parameter from the function definition, even if you don't use it.
        Returns:
            sensor_ranges   -- A column numpy array of the range values (meters)
            sensor_bearings -- A column numpy array of the bearings at which the sensor readings were taken (degrees)
                               The bearing values are not used in the Localization module, so you may return a empty numpy array
        """
        while self.is_done_collecting == False:
            await asyncio.sleep(0.1)

        LOG.info("Collecting Data")
        self.ble.send_command(CMD.SEND_PID_DATA, "")


        await asyncio.sleep(5)

        LOG.info('Finished receiving data.')
        LOG.info(self.results)

        imu_angles_deg = np.array([i[1] for i in self.results])
        tof_readings = np.array([i[3] for i in self.results])

        trials = 1

        angle = imu_angles_deg.reshape(
            (18, trials)).mean(axis=1)[np.newaxis].T
        distance = tof_readings.reshape((18, trials)).mean(axis=1)[
            np.newaxis].T / 1000  # mm -> m

        # save to csv file
        angle = '(5,3)'
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"robot_data_{timestamp}_{angle}.csv"

        with open(filename, 'w', newline='') as csvfile:
            csvwriter = csv.writer(csvfile)
            # Write header
            csvwriter.writerow(['Time (ms)', 'IMU', 'PWM', 'TOF'])
            # Write all data rows
            csvwriter.writerows(self.results)

        print(f"Data saved to {filename}")

        return distance, angle

In [138]:
# Get ArtemisBLEController object
ble = get_ble_controller()

# Connect to the Artemis Device
ble.connect()

2025-05-14 13:53:26,979 | INFO     |: Looking for Artemis Nano Peripheral Device: c0:81:04:26:a3:64
2025-05-14 13:53:38,131 | INFO     |: Connected to c0:81:04:26:a3:64


In [66]:
ble.disconnect()

Exception: Not connected to a BLE device

In [136]:
# Initialize RealRobot with a Commander object to communicate with the plotter process
# and the ArtemisBLEController object to communicate with the real robot
robot = RealRobot(cmdr, ble)

# Initialize mapper
# Requires a VirtualRobot object as a parameter
mapper = Mapper(robot)

# Initialize your BaseLocalization object
# Requires a RealRobot object and a Mapper object as parameters
loc = Localization(robot, mapper)

## Plot Map
cmdr.plot_map()

2025-05-14 13:49:33,260 | INFO     |:  | Number of observations per grid cell: 18
2025-05-14 13:49:33,261 | INFO     |:  | Precaching Views...
2025-05-14 13:49:34,279 | INFO     |:  | Precaching Time: 1.017 secs
2025-05-14 13:49:34,280 | INFO     |: Initializing beliefs with a Uniform Distribution
2025-05-14 13:49:34,281 | INFO     |: Uniform Belief with each cell value: 0.00051440329218107


# Run an update step of the Bayes Filter

In [137]:
# Reset Plots
cmdr.reset_plotter()
ble.send_command(CMD.LOCALIZE, "")

# Init Uniform Belief
loc.init_grid_beliefs()

# Get Observation Data by executing a 360 degree rotation motion
await loc.get_observation_data()

# Run Update Step
loc.update_step()
loc.plot_update_step_data(plot_data=True)

# Plot Odom and GT
# current_odom, current_gt = robot.get_pose()
# cmdr.plot_gt(1.524, 0.9144)
# cmdr.plot_gt(current_gt[0], current_gt[1])
# cmdr.plot_odom(current_odom[0], current_odom[1])
# print(current_odom, current_gt)

2025-05-14 13:49:36,650 | INFO     |: Initializing beliefs with a Uniform Distribution
2025-05-14 13:49:36,650 | INFO     |: Uniform Belief with each cell value: 0.00051440329218107
2025-05-14 13:51:52,457 | INFO     |: Disconnected from A1C0DD95-B611-6CFA-AF80-6BCF50487F59


CancelledError: 

In [45]:
waypoints= [(-4, 3), (-2,-1), (1, -1), (2, -3), (5,-3), (5,-2), (0,3), (0,0)]

In [96]:
import math


def angle_to_turn(src, dst):
    """
    Given source and destination waypoints, return the angle (in radians)
    from the source to the destination, measured from the x-axis.

    Parameters:
        src: tuple of (x, y) - current position
        dst: tuple of (x, y) - target position

    Returns:
        angle in radians
    """
    dx = dst[0] - src[0]
    dy = dst[1] - src[1]
    return math.atan2(dy, dx)


# Example usage:
src = (-0.305, -1.219)
dst = (-0.6096, -0.3048)

angle = angle_to_turn(src, dst)

print(
    f"Angle to turn: {math.degrees(angle):.2f} degrees ({angle:.2f} radians)")

Angle to turn: 108.43 degrees (1.89 radians)


2025-05-14 13:09:59,402 | INFO     |: Disconnected from A1C0DD95-B611-6CFA-AF80-6BCF50487F59


In [ ]:
ble.send_command(CMD.LOCALIZE)

In [141]:
ble.send_command(CMD.SET_SETPOINT, 45)
ble.send_command(CMD.START_PID, "")

2025-05-14 13:56:20,340 | INFO     |: Disconnected from A1C0DD95-B611-6CFA-AF80-6BCF50487F59


In [140]:
ble.send_command(CMD.STOP_PID, "")

In [ ]:
# set setpint
ble.send_command(CMD.SET_SETPOINT, )